# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [8]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [9]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [10]:
# Import the necessary CSVs to Pandas DataFrames
sfo_file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(sfo_file_path)

mapbox_data = pd.read_csv(Path("Data/neighborhoods_coordinates.csv"))

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [11]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
sfo_group = sfo_data.groupby("year").mean().reset_index()

avg_housing = px.bar(
    sfo_group,
    x="year",
    y="housing_units",
    title="Average Housing Units per Year",
)

max_val=sfo_group['housing_units'].max()
min_val=sfo_group['housing_units'].min()

avg_housing.update_yaxes(range=[min_val, max_val])


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
sfo_group_no_units = sfo_group.drop(columns = "housing_units")

average_price_foot = px.line(
    sfo_group_no_units,
    x="year",
    y="sale_price_sqr_foot",
    title="Average Sale Price per Square Foot per Year",
)


def average_sales_price():
    """Average Sales Price Per Year."""
    
average_monthly_rent = px.line(
    sfo_group_no_units,
    x="year",
    y="gross_rent",
    title="Average Monthly Rent",
)


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
sfo_group_two = sfo_data.groupby(["year", "neighborhood"]).mean().reset_index().round(2)
price_per_foot = sfo_group_two.hvplot.line('year', 'sale_price_sqr_foot', groupby='neighborhood')


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

average_monthly_rent = sfo_group_two.hvplot.line('year', 'gross_rent', groupby='neighborhood')


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
price_vs_rent = sfo_group_two.hvplot.bar('year', ['gross_rent', 'sale_price_sqr_foot'], groupby = 'neighborhood')

    
def neighborhood_map():
    """Neighborhood Map."""

neighborhood_group = sfo_data.groupby("neighborhood").mean().reset_index()
joined_df = pd.concat([neighborhood_group, mapbox_data], axis='columns', join='inner')

scatter_map = px.scatter_mapbox(
    joined_df,
    lat="Lat",
    lon="Lon",
    color="neighborhood"
)

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [13]:
# Create a Title for the Dashboard
sfo_column = pn.Column(
    "## San Francisco Real Estate Comparisons by Neighborhood", housing_units_per_year(), average_gross_rent(), 
    average_sales_price(), average_price_by_neighborhood())

top_ten_column = pn.Column(
    "## Top 10 Most Expensive San Fransisco Neighborhoods Statistics", top_most_expensive_neighborhoods(), 
    most_expensive_neighborhoods_rent_sales())

map_column = pn.Column(
    "## San Francisco Real Estate Data with Map", neighborhood_map())

# Create a tab layout for the dashboard
sfo_dashboard = pn.Tabs(
    ("San Francisco Averages", sfo_column),
    ("Top Ten", top_ten_column),
    ("Map", map_column)
)

# Create the dashboard
sfo_dashboard

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Str(None)
        [2] Str(None)
        [3] Str(None)
        [4] Str(None)
    [1] Column
        [0] Markdown(str)
        [1] Str(None)
        [2] Str(None)
    [2] Column
        [0] Markdown(str)
        [1] Str(None)

## Serve the Panel Dashboard

In [14]:
# Serve the# dashboard
sfo_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Str(None)
        [2] Str(None)
        [3] Str(None)
        [4] Str(None)
    [1] Column
        [0] Markdown(str)
        [1] Str(None)
        [2] Str(None)
    [2] Column
        [0] Markdown(str)
        [1] Str(None)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()